In [13]:
import pandas as pd
import torch

In [14]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('''NumRooms,RoofType,Price
NA,NA,127500
2,NA,106000
4,Slate,178100
NA,NA,140000''')

In [15]:
data = pd.read_csv(data_file)
print(data)

   NumRooms RoofType   Price
0       NaN      NaN  127500
1       2.0      NaN  106000
2       4.0    Slate  178100
3       NaN      NaN  140000


由于目前的数据来看，RoofType只有两种情况，NaN和Slate，因此可以用true和false来表示

实际上get_dummies就是将数据转换成OneHot编码

In [16]:
inputs, targets = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       NaN           False          True
1       2.0           False          True
2       4.0            True         False
3       NaN           False          True


In [17]:
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       3.0           False          True
1       2.0           False          True
2       4.0            True         False
3       3.0           False          True


## practice

### 1. 删除缺失值数目最多的列

In [18]:
def delete_num_of_nan_column(df):
    max_nan_num = 0
    max_nan_col = ''
    for col in df.columns:
        print(col)
        col_nan = df[col].isna().sum()
        if col_nan > max_nan_num:
            max_nan_col = col
            max_nan_num = col_nan
    df.drop(max_nan_col, axis=1, inplace=True)
    
test_df = pd.read_csv(data_file)
print(test_df)
delete_num_of_nan_column(test_df)
print(test_df)


   NumRooms RoofType   Price
0       NaN      NaN  127500
1       2.0      NaN  106000
2       4.0    Slate  178100
3       NaN      NaN  140000
NumRooms
RoofType
Price
   NumRooms   Price
0       NaN  127500
1       2.0  106000
2       4.0  178100
3       NaN  140000


### 2. 将预处理后的数据集转换为张量格式

In [23]:
type(inputs)

pandas.core.frame.DataFrame

In [32]:
type(inputs.values)

numpy.ndarray

In [ ]:
inputs.values

In [25]:
torch.tensor(inputs.values)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [33]:
inputs.values.dtype

dtype('O')

当您遇到 "can't convert np.ndarray of type numpy.object_" 这样的错误时，通常是因为 NumPy 数组中包含了混合类型的数据或者复杂对象。这种情况下，NumPy 会将数组的 dtype（数据类型）设置为 'object'，这可能会导致某些转换操作失败。

可能的解决方案：

a. 使用 tolist() 方法：
将 NumPy 数组转换为 Python 列表，然后创建 DataFrame。
这种方法适用于简单的混合类型数组。

b. 创建单列 DataFrame：
将整个数组作为单一列添加到 DataFrame 中。
这保留了原始数据结构，但可能不适合需要多列数据的情况。

c. 预处理数据：
在转换之前处理特定类型的数据（例如，将列表转换为字符串）。
这种方法允许您根据具体需求自定义数据转换。

In [34]:
import numpy as np
import pandas as pd

# 创建一个包含混合类型的 NumPy 数组
mixed_array = np.array([1, 'two', 3.0, [4, 5]], dtype=object)

print("混合类型数组:")
print(mixed_array)
print("数组的数据类型:", mixed_array.dtype)

# 尝试转换为 DataFrame（这可能会引发错误）
try:
    df = pd.DataFrame(mixed_array)
    print("成功转换为 DataFrame:")
    print(df)
except Exception as e:
    print("转换失败:", str(e))

# 解决方案 1: 将数组转换为列表，然后创建 DataFrame
df_from_list = pd.DataFrame(mixed_array.tolist())
print("\n使用 tolist() 转换后的 DataFrame:")
print(df_from_list)

# 解决方案 2: 创建单列 DataFrame
df_single_column = pd.DataFrame({'mixed_data': mixed_array})
print("\n单列 DataFrame:")
print(df_single_column)

# 解决方案 3: 处理特定类型（例如，将列表转换为字符串）
processed_array = np.array([str(item) if isinstance(item, list) else item for item in mixed_array])
df_processed = pd.DataFrame(processed_array)
print("\n处理后的 DataFrame:")
print(df_processed)

混合类型数组:
[1 'two' 3.0 list([4, 5])]
数组的数据类型: object
成功转换为 DataFrame:
        0
0       1
1     two
2     3.0
3  [4, 5]

使用 tolist() 转换后的 DataFrame:
        0
0       1
1     two
2     3.0
3  [4, 5]

单列 DataFrame:
  mixed_data
0          1
1        two
2        3.0
3     [4, 5]

处理后的 DataFrame:
        0
0       1
1     two
2     3.0
3  [4, 5]


In [35]:
inputs.values.tolist()

[[3.0, False, True],
 [2.0, False, True],
 [4.0, True, False],
 [3.0, False, True]]

In [36]:
torch.tensor(inputs.values.tolist())

tensor([[3., 0., 1.],
        [2., 0., 1.],
        [4., 1., 0.],
        [3., 0., 1.]])